# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[0:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
chars[:100]

array([30, 50, 25, 18, 51, 79, 39, 38, 10, 77, 77, 77,  4, 25, 18, 18, 59,
       38, 52, 25, 35, 16, 82, 16, 79, 29, 38, 25, 39, 79, 38, 25, 82, 82,
       38, 25, 82, 16,  9, 79,  3, 38, 79, 81, 79, 39, 59, 38, 53,  5, 50,
       25, 18, 18, 59, 38, 52, 25, 35, 16, 82, 59, 38, 16, 29, 38, 53,  5,
       50, 25, 18, 18, 59, 38, 16,  5, 38, 16, 51, 29, 38, 26, 66,  5, 77,
       66, 25, 59, 17, 77, 77, 20, 81, 79, 39, 59, 51, 50, 16,  5], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
np.max(chars)+1

83

## Making training and validation batches

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the batch size. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [6]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the first split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 50)

In [8]:
train_x.shape

(10, 178650)

Looking at the size of this array, we see that we have rows equal to the batch size. When we want to get a batch out of here, we can grab a subset of this array that contains all the rows but has a width equal to the number of steps in the sequence. The first batch looks like this:

In [9]:
train_x[:,:50]

array([[30, 50, 25, 18, 51, 79, 39, 38, 10, 77, 77, 77,  4, 25, 18, 18, 59,
        38, 52, 25, 35, 16, 82, 16, 79, 29, 38, 25, 39, 79, 38, 25, 82, 82,
        38, 25, 82, 16,  9, 79,  3, 38, 79, 81, 79, 39, 59, 38, 53,  5],
       [38, 25, 35, 38,  5, 26, 51, 38, 19, 26, 16,  5, 19, 38, 51, 26, 38,
        29, 51, 25, 59, 73, 15, 38, 25,  5, 29, 66, 79, 39, 79, 68, 38,  1,
         5,  5, 25, 73, 38, 29, 35, 16, 82, 16,  5, 19, 73, 38, 21, 53],
       [81, 16,  5, 17, 77, 77, 15, 71, 79, 29, 73, 38, 16, 51, 27, 29, 38,
        29, 79, 51, 51, 82, 79, 68, 17, 38, 14, 50, 79, 38, 18, 39, 16, 56,
        79, 38, 16, 29, 38, 35, 25, 19,  5, 16, 52, 16, 56, 79,  5, 51],
       [ 5, 38, 68, 53, 39, 16,  5, 19, 38, 50, 16, 29, 38, 56, 26,  5, 81,
        79, 39, 29, 25, 51, 16, 26,  5, 38, 66, 16, 51, 50, 38, 50, 16, 29,
        77, 21, 39, 26, 51, 50, 79, 39, 38, 66, 25, 29, 38, 51, 50, 16],
       [38, 16, 51, 38, 16, 29, 73, 38, 29, 16, 39, 57, 15, 38, 29, 25, 16,
        68, 38, 51, 50, 

I'll write another function to grab batches out of the arrays made by `split_data`. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [10]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

## Building the model

Below is a function where I build the graph for the network.

In [14]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
    
    # When we're using this network for sampling later, we'll be passing in
    # one character at a time, so providing an option for that
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # One-hot encoding the input and target characters
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)

    ### Build the RNN layers
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(lstm_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)

    ### Run the data through the RNN layers
    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    final_state = state
    
    # Reshape output so it's a bunch of rows, one output row for each step for each batch
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    # Now connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and batch
    logits = tf.matmul(output, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    preds = tf.nn.softmax(logits, name='predictions')
    
    # Reshape the targets to match the logits
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)

    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes
    # NOTE: I'm using a namedtuple here because I think they are cool
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to write it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [15]:
batch_size = 10
num_steps = 100 
lstm_size = 512
num_layers = 2
learning_rate = 0.001
keep_prob = 0.5

## Training

Time for training which is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}_v{validation loss}.ckpt`

In [16]:
epochs = 20
# Save every N iterations
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/i{}_l{}_v{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/20  Iteration 1/35720 Training loss: 4.4208 1.3842 sec/batch
Epoch 1/20  Iteration 2/35720 Training loss: 4.3823 0.2067 sec/batch
Epoch 1/20  Iteration 3/35720 Training loss: 4.2444 0.2188 sec/batch
Epoch 1/20  Iteration 4/35720 Training loss: 4.5646 0.2144 sec/batch
Epoch 1/20  Iteration 5/35720 Training loss: 4.4444 0.2112 sec/batch
Epoch 1/20  Iteration 6/35720 Training loss: 4.2743 0.2122 sec/batch
Epoch 1/20  Iteration 7/35720 Training loss: 4.1526 0.2166 sec/batch
Epoch 1/20  Iteration 8/35720 Training loss: 4.0538 0.2203 sec/batch
Epoch 1/20  Iteration 9/35720 Training loss: 3.9771 0.2096 sec/batch
Epoch 1/20  Iteration 10/35720 Training loss: 3.9067 0.2095 sec/batch
Epoch 1/20  Iteration 11/35720 Training loss: 3.8500 0.2124 sec/batch
Epoch 1/20  Iteration 12/35720 Training loss: 3.8004 0.2186 sec/batch
Epoch 1/20  Iteration 13/35720 Training loss: 3.7504 0.2157 sec/batch
Epoch 1/20  Iteration 14/35720 Training loss: 3.7061 0.2066 sec/batch
Epoch 1/20  Iteration 15/3572

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [17]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i35720_l512_v1.670.ckpt"
all_model_checkpoint_paths: "checkpoints/i16000_l512_v1.399.ckpt"
all_model_checkpoint_paths: "checkpoints/i16200_l512_v1.385.ckpt"
all_model_checkpoint_paths: "checkpoints/i16400_l512_v1.404.ckpt"
all_model_checkpoint_paths: "checkpoints/i16600_l512_v1.400.ckpt"
all_model_checkpoint_paths: "checkpoints/i16800_l512_v1.388.ckpt"
all_model_checkpoint_paths: "checkpoints/i17000_l512_v1.411.ckpt"
all_model_checkpoint_paths: "checkpoints/i17200_l512_v1.402.ckpt"
all_model_checkpoint_paths: "checkpoints/i17400_l512_v1.407.ckpt"
all_model_checkpoint_paths: "checkpoints/i17600_l512_v1.423.ckpt"
all_model_checkpoint_paths: "checkpoints/i17800_l512_v1.428.ckpt"
all_model_checkpoint_paths: "checkpoints/i18000_l512_v1.418.ckpt"
all_model_checkpoint_paths: "checkpoints/i18200_l512_v1.426.ckpt"
all_model_checkpoint_paths: "checkpoints/i18400_l512_v1.427.ckpt"
all_model_checkpoint_paths: "checkpoints/i18600_l512_v1.429.ckpt"
all_model_check

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [19]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [20]:
checkpoint = "checkpoints/i35720_l512_v1.670.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i35720_l512_v1.670.ckpt
FardraiI and the
next day to be where and institutions of the worries in the morning from
the open chasb that had always had already with the tass of poverty
in me of people in the air, in ready to begin telling Stepan Arkadyevitch
at the races; but to this fearful movement interview with wime, that
had been so cleverly and happy, and that he was to see which had been that
of everymonot public opinion was not what, but they would be a growing
heavenly flirting on.

When the third day Sprenthy missham affectionations supporty over his
man before the tiny which he liked to see him. But after a previous day
and was taken off, the merest in the closest recognizing him with smiles
that he had been felt by moneys as he had seen now, and half times in
the same special row forests of the peasantry and his self-conformed expent
with him, and with his characteristic step to move his self-confidence
of his own, and he h

In [21]:
checkpoint = "checkpoints/i35720_l512_v1.670.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i35720_l512_v1.670.ckpt
Farth, while he
does not think it. On the feeling, and to show men and means so triedered even
of hisself.

"It's not though," said Alexey Alexandrovitch.

Sergey Ivanovitch felt that the committee which she could, too, there
was the count, the bailiff had an ecstasy of speaking in the air,
order the world, which was that sort of himself and irritating the president
was transformed, but a sign of stress one of the artilest, as it were
apoled to all dutgs of skate.

It was infend why there was a mistake as could in the side paction. It must be
still all done of what sand all during in his own thing Iron wound that
he carefully began to prove any complex it only flung and eager to be side to
doubt. Levin, lean him that a man of the powerful source, he would have
caused her. She turned the conversation with her things.

"Now masters, and there's a little right to make their position."

"Anna, it is not good for 

In [22]:
checkpoint = "checkpoints/i35720_l512_v1.670.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="The")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i35720_l512_v1.670.ckpt
The Amerin-ble the
impressions of the morning. As in that case, he felt that the immortange
always tried and take past his pockets, with a sudden and talking
to the platform read it. And he felt that the complexing each otherse
porter she had foreal its hind by such dusting into which he was complete
controlless and accounts of the world, and that the master of the
awful position with Anna Arkadyevna used to say to him something about so
mangumed and important to himself, and even the fact that she was feeling the
same intense for something. And all this tender or ten stop to the railway
snalformence of the night ran down to him there to the message of his place
with a friendly accidnation. With the president, this position, the slightest
definite, angles, was that it was one of the province, but simply
shaded acquaintances. And so immediately she might do not to do.

Talking to himself his own past, and she 

In [23]:
checkpoint = "checkpoints/i35720_l512_v1.670.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="happy")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i35720_l512_v1.670.ckpt
happy?
He thought so much clearers, and what she had not been that seppt show
it all in the sense that their former cause of the meetings of the alternal. He
was confided by this topic of tention, and the female visitors
were engrossed and he had a great sense of rivalry in the country, as the
man's care of the peasantry had been bad for not build and a sense of inonisation,
and a sense of terror and the necks, and forced, and as he always
wroteed in with other machines from even before the meeting said, against him
to be sent all to forget it all, and there were some hand by the
aim of the meadow acred and thrown that men in the most surraund
and the necks incident. She felt she was seeing a delightful summer at the
elections.

"Anyoner returned, darling, sweet all her skirt. I liked you," he
said all he felt afraid.

She glanced at him as though he were as glad of an unmistakable
scat..

"Will I hair hamp?"